In [19]:
from pymongo import MongoClient

In [20]:
from dotenv import load_dotenv
import os
load_dotenv()
ATLAS_KEY= os.getenv('ATLAS_KEY')

client= MongoClient(ATLAS_KEY)

db=client.Top250


# Quel est le film le plus long?

In [21]:
pipeline = [
    {
        "$sort": {"duree": -1}
    },
    {
        "$limit": 1
    },
    {
        "$project": {"_id": 0, "titre": 1}
    }
    
]

result = db.Top_films.aggregate(pipeline)

for i in result:
    i=i['titre'][0]
    print(f"Le film le plus long est {i}")

Le film le plus long est Gone with the Wind


# Quels sont les 5 films les mieux notés?

In [22]:
pipeline = [
    {
        "$sort": {"note": -1}
    },
    {
        "$limit": 5
    },
    {
        "$project": {"_id": 0, "titre": 1, "note":1}
    }
]

result = db.Top_films.aggregate(pipeline)
for index, film in enumerate(result,1):
    titre= film['titre'][0]
    note=film['note'][0]
    film_info = f"le n° {index} est {titre}: note de {note}"
    print(film_info)


le n° 1 est The Shawshank Redemption: note de 9.3
le n° 2 est The Godfather: note de 9.2
le n° 3 est The Lord of the Rings: The Return of the King: note de 9.0
le n° 4 est The Godfather Part II: note de 9.0
le n° 5 est The Dark Knight: note de 9.0


# Dans combien de films a joué Morgan Freeman?

In [24]:
pipeline = [
    {
        "$match": {
            "casting": "Morgan Freeman"
        }
    },
  
    {
        "$group": {
            "_id": None,
            "count": {
                "$sum": 1
            },
            "titres": {
                "$push": "$titre"
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "count": 1,
            "titres": 1
        }
    }
]

result = db.Top_films.aggregate(pipeline)

for i in result:
    compte = i['count']
    titres = [t[0] for t in i['titres']]
    
    print(f"Morgan Freeman a joué dans {compte} films: {', '.join(titres)}")

Morgan Freeman a joué dans 4 films: The Shawshank Redemption, Million Dollar Baby, Unforgiven, Se7en


# Dans combien de films a joué Tom Cruise?

In [25]:
pipeline = [
    {
        "$match": {
            "casting": "Tom Cruise"
        }
    },
  
    {
        "$group": {
            "_id": None,
            "count": {
                "$sum": 1
            },
            "titres": {
                "$push": "$titre"
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "count": 1,
            "titres": 1
        }
    }
]

result = db.Top_films.aggregate(pipeline)

for i in result:
    compte=i['count']
    titres=[i for i in i['titres']][0][0]
  
    print(f"Tom Cruise a joué dans {compte} film {titres}")

Tom Cruise a joué dans 1 film Top Gun: Maverick


# Quels sont les 3 meilleurs films par genre?

In [31]:
films_horreur = db.Top_films.find({"genre": "Horror"}).sort([("note", -1)]).limit(3)
print("Les 3 meilleurs films d'horreur sont :")
for film in films_horreur:
    print(film["titre"][0], "-", film["note"][0])
print()

films_drame = db.Top_films.find({"genre": "Drama"}).sort([("note", -1)]).limit(3)
print("Les 3 meilleurs films dramatiques sont :")
for film in films_drame:
    print(film["titre"][0], "-", film["note"][0])
print()

films_comique = db.Top_films.find({"genre": "Comedy"}).sort([("note", -1)]).limit(3)
print("Les 3 meilleurs films comiques sont :")
for film in films_comique:
    print(film["titre"][0], "-", film["note"][0])

Les 3 meilleurs films d'horreur sont :
Psycho - 8.5
Alien - 8.5
The Shining - 8.4

Les 3 meilleurs films dramatiques sont :
The Shawshank Redemption - 9.3
The Godfather - 9.2
The Dark Knight - 9.0

Les 3 meilleurs films comiques sont :
Life Is Beautiful - 8.6
The Intouchables - 8.5
Modern Times - 8.5


# Parmis les 100 meilleurs films, quel pourcentage est francais? américain?

In [28]:

pipeline = [
    {
        "$sort": {"note": -1}                                                          #on classe la collection par nombre decroissant de notation
    },
    {
        "$limit": 100                                                                  # se limite aux 100 premiers rslts
    },
    {
        "$match": {"pays": {"$in": ["United States"]}}                                 #garde les pays dans lequel film d'origine, de co-origine "france"
    },
    {
        "$group": {
            "_id": None,
            "count": {"$sum": 1}                                                       #on compte ces pays
        }
    },
    {   "$project":{                                                                  #garde que le champ du comptage de film francais
                "_id":0, "count":1
    }}
]

result = db.Top_films.aggregate(pipeline)
for i in result:
    for v,x in i.items():
        print (f"le pourcentage de film francais est de {x}%")
        
        
pipeline = [
    {
        "$sort": {"note": -1}
    },
    {
        "$limit": 100
    },
    {
        "$match": {"pays": {"$in": ["United States"]}}
    },
    {
        "$group": {
            "_id": None,
            "count": {"$sum": 1}
        }
    },
    {   "$project":{
                "_id":0, "count":1
    }}
]

result = db.Top_films.aggregate(pipeline)
for i in result:
    for v,x in i.items():
        print (f"le pourcentage de film americain est de {x}%")


le pourcentage de film francais est de 76%
le pourcentage de film americain est de 76%


# Par genre, quelle est la durée moyenne des films?

In [29]:
pipeline = [
    {"$unwind": "$genre"},                              #va sélectionner tous les champs uniques de genre, va décomposer le tableau en autant de genres qu'il existe
    {
        "$group": {
            "_id": "$genre",
            "avg_duration": {"$avg": "$duree"}
        }
    },
     {
        "$project": {
             "_id": 0,
            "genre": "$_id",
            "avg_duration": {"$round": ["$avg_duration", 0]}
        }}
]
result = db.Top_films.aggregate(pipeline)

for document in result:
    values = list(document.values())
    for i in range(0, len(values), 2):
        print(f"le genre {values[i]} a une durée moyenne de  {int(values[i+1])} minutes")

le genre Crime a une durée moyenne de  132 minutes
le genre Mystery a une durée moyenne de  125 minutes
le genre Comedy a une durée moyenne de  106 minutes
le genre Biography a une durée moyenne de  146 minutes
le genre Adventure a une durée moyenne de  130 minutes
le genre Family a une durée moyenne de  115 minutes
le genre Sport a une durée moyenne de  128 minutes
le genre History a une durée moyenne de  152 minutes
le genre Drama a une durée moyenne de  134 minutes
le genre Musical a une durée moyenne de  103 minutes
le genre Romance a une durée moyenne de  118 minutes
le genre Thriller a une durée moyenne de  124 minutes
le genre Horror a une durée moyenne de  121 minutes
le genre Action a une durée moyenne de  134 minutes
le genre Film-Noir a une durée moyenne de  110 minutes
le genre Sci-Fi a une durée moyenne de  132 minutes
le genre War a une durée moyenne de  134 minutes
le genre Animation a une durée moyenne de  101 minutes
le genre Music a une durée moyenne de  134 minutes
l

# Liste des films classés par genre selon critère de durée

In [32]:
genres = db.Top_films.distinct("genre")
for genre in genres:
    films = db.Top_films.find({"genre": genre}).sort("duree", -1).limit(3)
    print(f"Les 3 films les plus longs du genre {genre}:")
    for film in films:
        print(film["titre"][0], "-", film["duree"], "minutes")
    print()

Les 3 films les plus longs du genre Action:
Seven Samurai - 207 minutes
The Lord of the Rings: The Return of the King - 201 minutes
Avengers: Endgame - 181 minutes

Les 3 films les plus longs du genre Adventure:
Lawrence of Arabia - 218 minutes
Ben-Hur - 212 minutes
The Lord of the Rings: The Return of the King - 201 minutes

Les 3 films les plus longs du genre Animation:
Princess Mononoke - 134 minutes
Spirited Away - 125 minutes
Howl's Moving Castle - 119 minutes

Les 3 films les plus longs du genre Biography:
Lawrence of Arabia - 218 minutes
Schindler's List - 195 minutes
The Wolf of Wall Street - 180 minutes

Les 3 films les plus longs du genre Comedy:
The Wolf of Wall Street - 180 minutes
3 Idiots - 170 minutes
Green Book - 130 minutes

Les 3 films les plus longs du genre Crime:
Once Upon a Time in America - 229 minutes
The Godfather Part II - 202 minutes
The Green Mile - 189 minutes

Les 3 films les plus longs du genre Drama:
Gone with the Wind - 238 minutes
Once Upon a Time in A

## Quels sont les séries les mieux notées ?


In [34]:
series_mieux_notes = db.Top_series.find().sort([("score", -1)]).limit(5)
print('Les séries les mieux notées sont : ')
for serie in series_mieux_notes:
    print(serie["title"][0], "-", serie["score"])

Les séries les mieux notées sont : 
Bluey - 9.5
Planet Earth II - 9.5
Breaking Bad - 9.5
Band of Brothers - 9.4
Planet Earth - 9.4
